
# 🌸 MLflow Taller — Clasificación con *Iris* (Ejecución **local** en VS Code)

**Objetivos**  
- Aplicar **MLflow Tracking** para registrar parámetros, métricas y modelos.  
- Usar `mlflow.sklearn.log_model()` con **firma** (`infer_signature`) y **`input_example`**.  
- Visualizar los experimentos desde la **UI de MLflow** en `http://127.0.0.1:5000`.  
- Comprender un **flujo reproducible** de un experimento supervisado.

> Este cuaderno está pensado para ejecutarse **localmente** en **Visual Studio Code** (extensión Jupyter).
> Si quieres Model Registry completo, usa el **Tracking Server** con SQLite (celda opcional más abajo).



## ⚙️ Preparación rápida en VS Code (recomendado)
1. Crea y activa un entorno virtual en tu proyecto:
   - **Windows (PowerShell):**
     ```powershell
     py -m venv .venv
     .\.venv\Scripts\Activate.ps1
     ```
   - **macOS / Linux (bash/zsh):**
     ```bash
     python3 -m venv .venv
     source .venv/bin/activate
     ```
2. Abre este `.ipynb` en VS Code y **selecciona el intérprete** de `.venv` como kernel.
3. Ejecuta las celdas en orden.


In [ ]:

# 🧩 Instalación de dependencias en el **kernel actual**
import sys, subprocess
pkgs = ["mlflow==2.14.1", "scikit-learn==1.5.2", "matplotlib==3.9.2", "pandas==2.2.2", "numpy"]
subprocess.check_call([sys.executable, "-m", "pip", "install", *pkgs])
print("✔️ Dependencias instaladas en:", sys.executable)


In [ ]:

# 🚀 Configuración MLflow (File Store local ./mlruns)
import os, json, time, sys
from pathlib import Path
import mlflow
import mlflow.sklearn

TRACKING_DIR = Path("mlruns").resolve()
TRACKING_DIR.mkdir(parents=True, exist_ok=True)

# Usamos URI tipo file://... de manera portable (Windows/macOS/Linux)
TRACKING_URI_FILE = TRACKING_DIR.as_uri()
mlflow.set_tracking_uri(TRACKING_URI_FILE)

EXPERIMENT_NAME = "MLflow Iris Classification Demo (Local)"
mlflow.set_experiment(EXPERIMENT_NAME)

print("MLflow version:", mlflow.__version__)
print("Tracking URI (file):", mlflow.get_tracking_uri())
print("Experiment:", EXPERIMENT_NAME)


In [ ]:

# 📦 Carga de datos y división
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris(as_frame=True)
X = iris.data.copy()
y = iris.target.copy()
X.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("X shape:", X.shape, "| y shape:", y.shape)
X.head()


In [ ]:

# 🏋️‍♀️ Entrenamiento + Tracking con MLflow
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from mlflow.models.signature import infer_signature

params = {
    "C": 1.0,
    "max_iter": 200,
    "solver": "lbfgs",
    "multi_class": "auto",
    "random_state": 42
}

with mlflow.start_run() as run:
    run_id = run.info.run_id
    print("Run ID:", run_id)

    model = LogisticRegression(
        C=params["C"],
        max_iter=params["max_iter"],
        solver=params["solver"],
        multi_class=params["multi_class"],
        random_state=params["random_state"]
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1m = f1_score(y_test, y_pred, average="macro")
    print(f"Accuracy: {acc:.4f} | F1-macro: {f1m:.4f}")

    # Log de hiperparámetros y métricas
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("f1_macro", f1m)

    # Matriz de confusión (como imagen)
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation="nearest")
    ax.set_title("Confusion Matrix (Iris)")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_xticks([0,1,2]); ax.set_yticks([0,1,2])
    ax.set_xticklabels(iris.target_names); ax.set_yticklabels(iris.target_names)
    for (i, j), v in np.ndenumerate(cm):
        ax.text(j, i, str(v), ha='center', va='center')
    fig.colorbar(im)
    plt.tight_layout()
    mlflow.log_figure(fig, "confusion_matrix.png")
    plt.close(fig)

    # Reporte de clasificación (artefacto JSON)
    report = classification_report(y_test, y_pred, target_names=iris.target_names, output_dict=True)
    import json
    with open("classification_report.json", "w") as f:
        json.dump(report, f, indent=2)
    mlflow.log_artifact("classification_report.json")

    # Firma e input_example
    from mlflow.models.signature import infer_signature
    signature = infer_signature(X_test, model.predict(X_test))
    input_example = X_test.iloc[[0]]

    # Intento de registro (si hay Registry); si no, log del artefacto
    registered_name = "iris-logreg-mlflow"
    try:
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=signature,
            input_example=input_example,
            registered_model_name=registered_name
        )
        print(f"Modelo registrado como '{registered_name}' (si el backend lo permite).")
    except Exception as e:
        print("Model Registry no disponible en file store. Se guarda como artefacto del run.")
        mlflow.sklearn.log_model(
            sk_model=model,
            artifact_path="model",
            signature=signature,
            input_example=input_example
        )

    # Guardamos el run_id para uso posterior
    with open("last_run_id_file_store.txt", "w") as f:
        f.write(run_id)

print("✅ Entrenamiento y logging finalizados en file store.")


In [ ]:

# 🖥️ Abrir la UI de MLflow (File Store) en http://127.0.0.1:5000
import subprocess, time, sys
from pathlib import Path

HOST = "127.0.0.1"
UI_PORT = 5000

# Nota: '--backend-store-uri' debe apuntar al directorio de 'mlruns' (no a la URI file://)
backend_path = str(Path("mlruns").resolve())

# Inicia la UI en segundo plano
ui_proc = subprocess.Popen([
    sys.executable, "-m", "mlflow", "ui",
    "--host", HOST,
    "--port", str(UI_PORT),
    "--backend-store-uri", backend_path
])

time.sleep(2)
print(f"🔗 Abre tu navegador en: http://{HOST}:{UI_PORT}")
print("⚠️ Mantén esta celda viva mientras uses la UI. Para detenerla: 'ui_proc.terminate()'.")



## 🧪 (Opcional) MLflow Tracking Server con SQLite (para **Model Registry**)
Si necesitas **Model Registry** completo de MLflow, en lugar de `mlflow ui` ejecuta un **Tracking Server** con base de datos SQLite.

- Se levantará en `http://127.0.0.1:5001`
- Backend: `sqlite:///mlflow.db`
- Artefactos: carpeta `./mlartifacts`

> Tras iniciar el servidor, **actualiza la `Tracking URI`** en tu código a `http://127.0.0.1:5001` y repite el entrenamiento para que el modelo se registre.


In [ ]:

# ▶️ Iniciar Tracking Server (SQLite) en 127.0.0.1:5001
import subprocess, sys, time
from pathlib import Path

HOST = "127.0.0.1"
SERVER_PORT = 5001

backend_db = Path("mlflow.db").resolve()
art_root = Path("mlartifacts").resolve()
art_root.mkdir(exist_ok=True, parents=True)

srv_proc = subprocess.Popen([
    sys.executable, "-m", "mlflow", "server",
    "--host", HOST,
    "--port", str(SERVER_PORT),
    "--backend-store-uri", f"sqlite:///{backend_db.as_posix()}",
    "--default-artifact-root", art_root.as_uri()  # file://... portable
])

time.sleep(3)
print(f"🔗 Tracking Server (con Registry): http://{HOST}:{SERVER_PORT}")
print("Ahora puedes apuntar MLflow a este servidor y re-entrenar para registrar en el Registry.")


In [ ]:

# 🔧 Cambiar la Tracking URI al servidor HTTP (Registry) y verificar
import mlflow
TRACKING_URI_SERVER = "http://127.0.0.1:5001"
mlflow.set_tracking_uri(TRACKING_URI_SERVER)
print("Tracking URI actual:", mlflow.get_tracking_uri())


In [ ]:

# 🔮 Cargar el modelo desde el Registry (si existe) o desde el último run del file store
import mlflow
import mlflow.pyfunc
from mlflow.tracking import MlflowClient
import pandas as pd

# Nuevos datos (Iris: 4 features)
new_data = pd.DataFrame([
    [5.1, 3.5, 1.4, 0.2],
    [6.7, 3.0, 5.2, 2.3]
], columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])

def load_model_with_fallback():
    # 1) Intentar desde el Registry en servidor HTTP (si se levantó)
    try:
        mlflow.set_tracking_uri("http://127.0.0.1:5001")
        client = MlflowClient()
        versions = client.get_latest_versions(name="iris-logreg-mlflow")
        if versions:
            latest = sorted(versions, key=lambda v: int(v.version))[-1]
            model_uri = f"models:/{latest.name}/{latest.version}"
            print("Cargando desde Model Registry:", model_uri)
            return mlflow.pyfunc.load_model(model_uri)
        else:
            print("Registry activo pero sin versiones del modelo. Se usará el file store...")
    except Exception as e:
        print("No se pudo acceder al Registry (¿servidor no iniciado?). Se usará el file store...")

    # 2) Fallback: cargar desde el último run del file store
    from pathlib import Path
    mlflow.set_tracking_uri(Path("mlruns").resolve().as_uri())
    with open("last_run_id_file_store.txt") as f:
        run_id = f.read().strip()
    model_uri = f"runs:/{run_id}/model"
    print("Cargando desde el run local:", model_uri)
    return mlflow.pyfunc.load_model(model_uri)

loaded_model = load_model_with_fallback()
preds = loaded_model.predict(new_data)

# Mapear índices a nombres de clase
from sklearn import datasets
iris = datasets.load_iris()
label_map = {i: name for i, name in enumerate(iris.target_names)}
pred_labels = [label_map[int(p)] for p in preds]

print("🆕 Nuevos datos:")
display(new_data)
print("🔎 Predicciones (índice):", preds.tolist())
print("🏷️ Predicciones (etiquetas):", pred_labels)



## 🛑 Detener procesos (cuando termines)
Ejecuta la celda siguiente para cerrar los servidores si siguen en ejecución.


In [ ]:

# Intenta terminar procesos si existen
try:
    ui_proc.terminate()  # UI de mlflow ui (5000)
except Exception:
    pass

try:
    srv_proc.terminate()  # Tracking Server (5001)
except Exception:
    pass

print("Procesos detenidos (si estaban activos).")



## ✅ Recap
- Logueaste **parámetros, métricas y artefactos** con MLflow en un **file store** local (`./mlruns`).  
- Levantaste la **UI de MLflow** en `http://127.0.0.1:5000`.  
- (Opcional) Iniciaste un **Tracking Server** con SQLite para habilitar **Model Registry** en `http://127.0.0.1:5001`.  
- Registraste un **modelo con firma e input_example** y realizaste **predicciones** con el modelo cargado desde el Registry o desde un run.

> Consejo: Ejecuta múltiples runs variando `C` y `max_iter` para comparar en la UI y seleccionar el mejor.
